In [7]:
%pwd
%cd d:\\Murgi\\code\\memes2024\\meme-research-2024

d:\Murgi\code\memes2024\meme-research-2024


d:\Murgi\code\memes2024\meme-research-2024\bertopic\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
import pandas as pd
import numpy as np
import pickle
import os
from tqdm.notebook import tqdm

In [9]:
templatebase_path = r'./data/processed/templatesbase.parquet'
templatebase = pd.read_parquet(templatebase_path)
templatebase

,clean_image_path,cpu_paths
0,D:/Memes2024_blurred/0-days-without-Lenny-Simp...,D:/Murgi/templatebase\0-days-without-Lenny-Sim...
1,D:/Memes2024_blurred/0-days-without-Lenny-Simp...,D:/Murgi/templatebase\0-days-without-Lenny-Sim...
2,D:/Memes2024_blurred/0-days-without-Lenny-Simp...,D:/Murgi/templatebase\0-days-without-Lenny-Sim...
3,D:/Memes2024_blurred/0-days-without-Lenny-Simp...,D:/Murgi/templatebase\0-days-without-Lenny-Sim...
4,D:/Memes2024_blurred/0-days-without-Lenny-Simp...,D:/Murgi/templatebase\0-days-without-Lenny-Sim...
...,...,...
36906,D:/Twitter2023_clean\2020_6-37918.jpg,D:/Murgi/templatebase\2020_6-37918.jpg
36907,D:/Twitter2023_clean\2020_6-41854.jpg,D:/Murgi/templatebase\2020_6-41854.jpg
36908,D:/Twitter2023_clean\2019_12-21203.jpg,D:/Murgi/templatebase\2019_12-21203.jpg
36909,D:/Twitter2023_clean\2020_6-21026.jpg,D:/Murgi/templatebase\2020_6-21026.jpg


In [10]:
meme_df = pd.read_parquet(r'D:\Murgi\code\memes2024\meme-research-2024\data\feature-matching\blurred_meme_df.parquet')
meme_df = meme_df.merge(templatebase, left_on='path', right_on='clean_image_path', how='inner')
meme_df.drop(columns=['clean_image_path'], inplace=True)
meme_df

,id,template_name,path,encoded_label,cpu_paths
0,0-days-without-Lenny-Simpsons_101,0-days-without-lenny-simpsons,D:/Memes2024_blurred/0-days-without-Lenny-Simp...,0,D:/Murgi/templatebase\0-days-without-Lenny-Sim...
1,0-days-without-Lenny-Simpsons_107,0-days-without-lenny-simpsons,D:/Memes2024_blurred/0-days-without-Lenny-Simp...,0,D:/Murgi/templatebase\0-days-without-Lenny-Sim...
2,0-days-without-Lenny-Simpsons_109,0-days-without-lenny-simpsons,D:/Memes2024_blurred/0-days-without-Lenny-Simp...,0,D:/Murgi/templatebase\0-days-without-Lenny-Sim...
3,0-days-without-Lenny-Simpsons_112,0-days-without-lenny-simpsons,D:/Memes2024_blurred/0-days-without-Lenny-Simp...,0,D:/Murgi/templatebase\0-days-without-Lenny-Sim...
4,0-days-without-Lenny-Simpsons_12,0-days-without-lenny-simpsons,D:/Memes2024_blurred/0-days-without-Lenny-Simp...,0,D:/Murgi/templatebase\0-days-without-Lenny-Sim...
...,...,...,...,...,...
33906,Zuckerberg_38,zuckerberg,D:/Memes2024_blurred/Zuckerberg\Zuckerberg_38.jpg,1144,D:/Murgi/templatebase\Zuckerberg_38.jpg
33907,Zuckerberg_39,zuckerberg,D:/Memes2024_blurred/Zuckerberg\Zuckerberg_39.jpg,1144,D:/Murgi/templatebase\Zuckerberg_39.jpg
33908,Zuckerberg_40,zuckerberg,D:/Memes2024_blurred/Zuckerberg\Zuckerberg_40.jpg,1144,D:/Murgi/templatebase\Zuckerberg_40.jpg
33909,Zuckerberg_5,zuckerberg,D:/Memes2024_blurred/Zuckerberg\Zuckerberg_5.jpg,1144,D:/Murgi/templatebase\Zuckerberg_5.jpg


In [11]:
X,y = meme_df['cpu_paths'], meme_df['template_name']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)

(27128,) (6783,)


Calc dists for the batch of 270 SM images.

In [12]:
sm_df = pd.read_parquet(r'D:\Murgi\code\meme-evaluator\v2\v2_predictions.parquet')
sm_df

blurred_image_folder = r'D:/SM270_blurred/Memes2022Final2'

blurred_imgs = os.listdir(blurred_image_folder)
blurred_imgs = [os.path.join(blurred_image_folder, meme) for meme in blurred_imgs]

# create df from list of meme paths
blurred_sm_df = pd.DataFrame(blurred_imgs, columns=["path"])
blurred_sm_df['id'] = blurred_sm_df['path'].apply(lambda x: os.path.splitext(os.path.basename(x))[0])
blurred_sm_df

merged = blurred_sm_df.merge(sm_df, on='id', how='inner')
merged.drop(columns=['path_y'], inplace=True)   
merged.rename(columns={'path_x':'path'}, inplace=True)

image_paths = merged['path'].tolist()
image_paths[:10]

['D:/SM270_blurred/Memes2022Final2\\2021.11.15_888.jpg',
 'D:/SM270_blurred/Memes2022Final2\\2021.11.20_426.png',
 'D:/SM270_blurred/Memes2022Final2\\2021.11.20_796.jpg',
 'D:/SM270_blurred/Memes2022Final2\\2021.11.24_558.jpg',
 'D:/SM270_blurred/Memes2022Final2\\2021.11.25_674.jpg',
 'D:/SM270_blurred/Memes2022Final2\\2021.11.27_357.jpg',
 'D:/SM270_blurred/Memes2022Final2\\2021.11.28_361.jpg',
 'D:/SM270_blurred/Memes2022Final2\\2021.11.29_519.jpg',
 'D:/SM270_blurred/Memes2022Final2\\2021.12.02_105.jpg',
 'D:/SM270_blurred/Memes2022Final2\\2021.12.02_190.jpg']

In [13]:
import cv2
import numpy as np
import os
from PIL import Image

def processimgs(template_target, templates, matchdir):
    if not os.path.exists(matchdir):
        os.makedirs(matchdir)

    try:
        fileloc1 = template_target
        im1 = Image.open(fileloc1)
        width, height = im1.size
        new_width = 800
        new_height = int((height * new_width) / width)  
        im1.draft('L',(new_width,new_height))
        img1 = np.asarray(im1) 

        orb = cv2.ORB_create()
        kp1, des1 = orb.detectAndCompute(img1, None)

    except:
        print(f'Error processing file: {template_target}')
        return
    if type(des1) is np.ndarray:
        done = []

        for r, d, f in os.walk(matchdir):
            for file in f:
                if '.txt' in file:
                    doneitem = os.path.splitext(file)[0]
                    done.append(doneitem)

        filename = os.path.basename(template_target)
        ext = os.path.splitext(filename)[1]
        processfileloc = os.path.join(matchdir, filename.replace(ext,'.txt'))
        with open (processfileloc, 'a') as f:
            matched_images = []
            FLANN_INDEX_LSH = 6
            index_params= dict(algorithm = FLANN_INDEX_LSH,
                    table_number = 6, # 12
                    key_size = 12,     # 20
                    multi_probe_level = 1) #2
            flann = cv2.FlannBasedMatcher(index_params)

            for template_compare in templates:
                if template_target != template_compare:
                    meme = os.path.splitext(os.path.basename(template_compare))[0] 
                    if meme not in done:
                        try:
                            fileloc2 = template_compare
                            if not os.path.exists(fileloc2):
                                print(f'File {fileloc2} does not exist')
                            im2 = Image.open(fileloc2)
                            # while keeping the aspect ratio calculate the new height if width is 800
                            width, height = im2.size
                            new_width = 800
                            new_height = int((height * new_width) / width)  
                            im2.draft('L',(new_width,new_height))
                            img2 = np.asarray(im2) 
                            kp2, des2 = orb.detectAndCompute(img2, None)

                            if type(des2) is np.ndarray:
                                matches = flann.match(des1, des2)
                                distances = []
                                if len(matches) > 0:
                                    for match in matches:
                                        distances.append(match.distance)
                                    matched_images.append((meme, distances))

                        except Exception as e:
                            print(f'Error processing {template_compare}: {e}')
                            raise e
            
            if len(matched_images) > 0:
                for meme, distances in matched_images:
                    f.write(f'{meme}\t')
                    f.write(','.join(map(str, distances))+ '\n')


In [14]:
X_train.shape

(27128,)

Calc distances for 270 sm images.

In [15]:
from tqdm.contrib.concurrent import  thread_map  # or thread_map

matchdir = r'D:\Murgi\code\memes2024\meme-research-2024\notebooks\feature-matching\distance-method-only\270-sm-dists'

thread_map(lambda template_target: processimgs(template_target, X_train, matchdir), image_paths)


  0%|          | 0/270 [00:00<?, ?it/s]

d:\Murgi\code\memes2024\meme-research-2024\bertopic\lib\site-packages\PIL\Image.py:3218: DecompressionBombWarning: Image size (91242942 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [14]:
import numpy as np

X_test_parts = np.array_split(X_test, 6)
y_test_parts = np.array_split(y_test, 6)

# create 6 dataframes
X_test_dfs = [pd.DataFrame({'path': X_test_parts[i], 'template_name': y_test_parts[i]}) for i in range(6)]

In [21]:
save_dir = r'D:\Murgi\code\memes2024\meme-research-2024\notebooks\feature-matching\distance-method-only\test-splits'

for i in range(6):
    X_test_dfs[i].to_parquet(os.path.join(save_dir, f'test_split_{i}.parquet'))

In [26]:
test_paths = X_test_dfs[0].path.tolist()

In [28]:
match_dir = "D:/imgflip-test-distances-part-0"
thread_map(lambda template_target: processimgs(template_target, X_train, matchdir=match_dir), test_paths)

  0%|          | 0/1131 [00:00<?, ?it/s]